In [1]:
#Imports
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy import ndimage
from skimage.filters import threshold_otsu
import tensorflow as tf
from keras import models, layers, preprocessing, optimizers
from sklearn.model_selection import train_test_split
import os
from keras import backend as K
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
from keras.models import load_model
from keras.utils.generic_utils import get_custom_objects

#VICTOR 

In [2]:
#use git repo to detect local directory
import git
repo = git.Repo('.', search_parent_directories=True)

In [31]:
#Preprocessing

def preProcessing(image_path):
    img = cv2.imread(image_path) #read image

    #to check for its aspect ratio
    height, width, channel = img.shape
    if width > height : #landscape
        img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
    
    #resize image to 256 x 512
    resizedImg = cv2.resize(img,(256, 512))
    # img_gray = cv2.cvtColor(resizedImg, cv2.COLOR_BGR2GRAY)
    img_filtered = cv2.GaussianBlur(resizedImg, (3,3), 0)
    return resizedImg

In [5]:
# img_edge = cv2.Canny(img_gray, 50, 100)
# (thresh, binImg) = cv2.threshold(img_filtered, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

In [32]:
# Output processed image into files

path_real = repo.working_tree_dir + "\\image\\REAL\\"
path_forge = repo.working_tree_dir + "\\image\\FORGE\\"

path_real_p = repo.working_tree_dir + "\\image_p\\REAL\\"
path_forge_p = repo.working_tree_dir + "\\image_p\\FORGE\\"

def preProc(path, path_p) :
    try:
        os_path = os.listdir(path)
        for img in os_path[:]:
            print("From : " + os.path.join(path, img))
            print("To : " + os.path.join(path_p, img))
            processed_img = preProcessing(os.path.join(path, img))
            status = plt.imsave(path_p + img, processed_img, cmap="gray")
            # status = cv2.imwrite(os.path.join(path_p, img), processed_img)
            print(status)
        return
    except Exception as e:
        print(f"Error : {e}")

In [33]:
preProc(path_real, path_real_p)
preProc(path_forge, path_forge_p)

From : e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image\REAL\IMG_8030.png
To : e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image_p\REAL\IMG_8030.png
None
From : e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image\REAL\IMG_8031.png
To : e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image_p\REAL\IMG_8031.png
None
From : e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image\REAL\IMG_8032.png
To : e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image_p\REAL\IMG_8032.png
None
From : e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image\REAL\IMG_8036.png
To : e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image_p\REAL\IMG_8036.png
None
From : e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image\REAL\IMG_8037.png
To : e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image_p\REAL\IMG_8037.png
None
From : e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image\REAL\IMG_8038.png
To : e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image_p\REAL\IMG_8038.png
None
From : e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image\REAL\IMG_8039.png
To : e:\

In [34]:
# Enumerate Label

label_name = ["REAL","FORGE"]

enumerated_label = {}

for i, class_label in enumerate(label_name) :
    enumerated_label[class_label] = i

enumerated_label

{'REAL': 0, 'FORGE': 1}

In [59]:
# LABELING

data = []
label = []
image_path = repo.working_tree_dir + "\\image_p\\"

def labeling(enumerated_label,image_path,data_img,label_img):
    for folder in os.listdir(image_path):
        label = enumerated_label[folder]
        for file in os.listdir(os.path.join(image_path, folder)):
            img_path = os.path.join(os.path.join(image_path, folder), file) 
            image = cv2.imread(img_path)
            data_img.append(image)
            label_img.append(label)

In [60]:
labeling(enumerated_label,image_path,data,label)

In [61]:
print(data)

[array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]

In [62]:
data = np.array(data, dtype = 'float')
label = np.array(label, dtype = 'float')

data.shape

# Reshape to 1 channel (Grayscale) <- If needed
# data = data.reshape(data.shape[0], data.shape[1], data.shape[2], 1)
# data.shape

(149, 512, 256, 3)

In [63]:
data = data/255.
print(data)

[[[[1.         1.         1.        ]
   [1.         1.         1.        ]
   [1.         1.         1.        ]
   ...
   [1.         1.         1.        ]
   [1.         1.         1.        ]
   [1.         1.         1.        ]]

  [[1.         1.         1.        ]
   [1.         1.         1.        ]
   [1.         1.         1.        ]
   ...
   [1.         1.         1.        ]
   [1.         1.         1.        ]
   [1.         1.         1.        ]]

  [[1.         1.         1.        ]
   [1.         1.         1.        ]
   [1.         1.         1.        ]
   ...
   [1.         1.         1.        ]
   [1.         1.         1.        ]
   [1.         1.         1.        ]]

  ...

  [[1.         1.         1.        ]
   [1.         1.         1.        ]
   [1.         1.         1.        ]
   ...
   [1.         1.         1.        ]
   [1.         1.         1.        ]
   [1.         1.         1.        ]]

  [[1.         1.         1.        ]
   [1. 

In [64]:
print(np.min(data))

0.0


In [65]:
print(label)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]


In [43]:
# MODEL , still figuring out
model = models.Sequential()

model.add(layers.Conv2D(32, (3,3), activation="relu",name='conv2d_1' ,
                        input_shape=(512,256,3))) #GrayScale :1 || RGB : 3
model.add(layers.MaxPooling2D((2,2), name="max_pooling2d_1"))
model.add(layers.Conv2D(64, (3,3), activation="relu",name='conv2d_2'))
model.add(layers.MaxPooling2D((2,2), name="max_pooling2d_2"))
model.add(layers.Conv2D(128, (3,3), activation="relu",name='conv2d_3'))
model.add(layers.MaxPooling2D((2,2), name="max_pooling2d_3"))
model.add(layers.Conv2D(256, (3,3), activation="relu",name='conv2d_4'))
model.add(layers.MaxPooling2D((2,2), name="max_pooling2d_4"))
model.add(layers.Conv2D(256, (3,3), activation="relu", name='conv2d_5'))
model.add(layers.MaxPooling2D((2,2), name="max_pooling2d_5"))
model.add(layers.Conv2D(512, (3,3), activation="relu",name='conv2d_6'))
model.add(layers.MaxPooling2D((2,2), name="max_pooling2d_6"))
model.add(layers.Flatten(name="flatten_1"))
model.add(layers.Dense(256, activation='relu', name="dense_1"))
model.add(layers.Dense(2, activation='relu', name="dense_2"))

In [32]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 510, 254, 32)      896       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 255, 127, 32)     0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 253, 125, 64)      18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 126, 62, 64)      0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 124, 60, 128)      73856     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 62, 30, 128)     

In [33]:
x_train, x_test, y_train, y_test = train_test_split(data, label, test_size=0.3, random_state=42)

In [66]:
res = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5)

Epoch 1/5
1/4 [======>.......................] - ETA: 2s - loss: 0.1584 - accuracy: 0.5000

KeyboardInterrupt: 

In [93]:
model.evaluate(x_test, y_test)

2/2 [==============================] - 3s 1s/step - loss: 0.6931 - accuracy: 0.6889


[0.6931471824645996, 0.6888889074325562]

In [122]:
def center_normalize(x):
    return (x - K.mean(x)) / K.std(x)

def root_mean_squared_error(y_true, y_pred):
    y_pred = tf.cast(y_pred, tf.float32)
    y_true = tf.cast(y_true, tf.float32)
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

model = models.Sequential()

get_custom_objects().update({'center_normalize': center_normalize, 'root_mean_squared_error': root_mean_squared_error})
model.add(layers.Activation(activation='center_normalize', input_shape=(512, 256, 3)))

model.add(layers.Convolution2D(64, 3, 3, padding='same'))
model.add(layers.Activation('relu'))
model.add(layers.Convolution2D(64, 3, 3, padding='valid'))
model.add(layers.Activation('relu'))
model.add(layers.ZeroPadding2D(padding=(1, 1)))
model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(layers.Dropout(0.25))

# model.add(layers.Convolution2D(96, 3, 3, padding='same'))
# model.add(layers.Activation('relu'))
# model.add(layers.Convolution2D(96, 3, 3, padding='valid'))
# model.add(layers.Activation('relu'))
# model.add(layers.ZeroPadding2D(padding=(1, 1)))
# model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
# model.add(layers.Dropout(0.25))

model.add(layers.Convolution2D(128, 2, 2, padding='same'))
model.add(layers.Activation('relu'))
model.add(layers.Convolution2D(128, 2, 2, padding='same'))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Flatten())
model.add(layers.Dense(2048))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.5))


model.add(layers.Dense(2))
model.add(layers.Activation('sigmoid'))

model.summary()


Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 activation_71 (Activation)  (None, 512, 256, 3)       0         
                                                                 
 conv2d_32 (Conv2D)          (None, 171, 86, 64)       1792      
                                                                 
 activation_72 (Activation)  (None, 171, 86, 64)       0         
                                                                 
 conv2d_33 (Conv2D)          (None, 57, 28, 64)        36928     
                                                                 
 activation_73 (Activation)  (None, 57, 28, 64)        0         
                                                                 
 zero_padding2d_8 (ZeroPaddi  (None, 59, 30, 64)       0         
 ng2D)                                                           
                                                     

In [123]:
adam = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=adam, loss='root_mean_squared_error', metrics=['accuracy'])

In [124]:
x_train, x_test, y_train, y_test = train_test_split(data, label, test_size=0.3, random_state=5)

In [125]:
early_stopping = EarlyStopping(monitor='val_loss', patience=4, verbose=1, mode='auto')
res = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, callbacks=[early_stopping])

Epoch 1/100
4/4 [==============================] - 3s 551ms/step - loss: 0.5033 - accuracy: 0.6346 - val_loss: 0.4744 - val_accuracy: 0.6000
Epoch 2/100
4/4 [==============================] - 2s 444ms/step - loss: 0.4588 - accuracy: 0.4615 - val_loss: 0.4415 - val_accuracy: 0.4222
Epoch 3/100
4/4 [==============================] - 2s 435ms/step - loss: 0.4311 - accuracy: 0.5000 - val_loss: 0.4107 - val_accuracy: 0.5556
Epoch 4/100
4/4 [==============================] - 2s 458ms/step - loss: 0.3981 - accuracy: 0.4712 - val_loss: 0.3844 - val_accuracy: 0.7333
Epoch 5/100
4/4 [==============================] - 2s 441ms/step - loss: 0.3802 - accuracy: 0.5577 - val_loss: 0.3653 - val_accuracy: 0.7111
Epoch 6/100
4/4 [==============================] - 2s 439ms/step - loss: 0.3587 - accuracy: 0.6154 - val_loss: 0.3487 - val_accuracy: 0.4889
Epoch 7/100
4/4 [==============================] - 2s 450ms/step - loss: 0.3470 - accuracy: 0.5385 - val_loss: 0.3362 - val_accuracy: 0.4222
Epoch 8/100
4

In [126]:
score = model.evaluate(x_test, y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

2/2 [==============================] - 0s 72ms/step - loss: 0.0949 - accuracy: 0.6222
Test loss: 0.09490445256233215
Test accuracy: 0.6222222447395325


In [53]:
# Generate a prediction using model.predict() 
# and calculate it's shape:
print("Generate a prediction")
prediction = model.predict_on_batch(x_test)
print("prediction shape:", prediction.shape)
print(prediction)

Generate a prediction
prediction shape: (45, 2)
[[8.7050190e-05 7.7860983e-05]
 [9.9972683e-01 9.9973345e-01]
 [9.9961579e-01 9.9953991e-01]
 [9.9947894e-01 9.9914080e-01]
 [9.9928761e-01 9.9908870e-01]
 [9.9760032e-01 9.9609947e-01]
 [8.3799459e-05 9.6914438e-05]
 [4.4369194e-01 3.1848848e-01]
 [2.6107132e-03 2.1170080e-03]
 [9.9990177e-01 9.9978995e-01]
 [9.9592602e-01 9.9442750e-01]
 [9.9973869e-01 9.9952400e-01]
 [9.9867404e-01 9.9703586e-01]
 [9.7523367e-01 9.6608347e-01]
 [9.9945521e-01 9.9858069e-01]
 [9.9986303e-01 9.9981821e-01]
 [9.9866378e-01 9.9841672e-01]
 [9.9996662e-01 9.9996907e-01]
 [4.7143102e-03 3.9515495e-03]
 [2.0489097e-04 3.0136108e-04]
 [9.9996829e-01 9.9996376e-01]
 [9.9933302e-01 9.9888742e-01]
 [9.9940646e-01 9.9921477e-01]
 [9.9998963e-01 9.9998695e-01]
 [9.9951690e-01 9.9945301e-01]
 [9.9930865e-01 9.9900955e-01]
 [9.9991208e-01 9.9989152e-01]
 [9.9988723e-01 9.9977100e-01]
 [9.9967432e-01 9.9967980e-01]
 [9.9269539e-01 9.8687184e-01]
 [9.9997753e-01 9.9995

In [127]:
model.save("model.h5")

In [6]:
def center_normalize(x):
    return (x - K.mean(x)) / K.std(x)

def root_mean_squared_error(y_true, y_pred):
    y_pred = tf.cast(y_pred, tf.float32)
    y_true = tf.cast(y_true, tf.float32)
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

adam = tf.keras.optimizers.Adam(learning_rate=0.0001)

model = load_model('model.h5', custom_objects={'center_normalize': layers.Activation(center_normalize),'root_mean_squared_error': root_mean_squared_error})
model.compile(optimizer=adam, loss=root_mean_squared_error, metrics=['accuracy'])

In [19]:
# Forge
#path_test = repo.working_tree_dir + "\\image_t\\IMG_8103.png"
#path_test = repo.working_tree_dir + "\\image_t\\IMG_8122.png"
#path_test = repo.working_tree_dir + "\\image_t\\IMG_8137.png"
#path_test = repo.working_tree_dir + "\\image_t\\IMG_8139.png"
path_test = repo.working_tree_dir + "\\image_t\\IMG_8149.png"

# Real
#path_test = repo.working_tree_dir + "\\image_t\\IMG_8048.png"
#path_test= repo.working_tree_dir + "\\image_t\\IMG_8061.png"
#path_test= repo.working_tree_dir + "\\image_t\\IMG_8079.png"
#path_test= repo.working_tree_dir + "\\image_t\\IMG_8054.png"
#path_test= repo.working_tree_dir + "\\image_t\\IMG_8085.png"
#path_test= repo.working_tree_dir + "\\image_t\\IMG_8044.png"

img = cv2.imread(path_test)

#to check for its aspect ratio 
height, width, channel = img.shape
if width > height : #landscape
    img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)

#resize image to 256 x 512
img = cv2.resize(img,(512, 256))
img = np.reshape(img,[1,512,256,3])

predictions = model.predict(img)

print(predictions)

res = (predictions[0][0] + predictions[0][1])/2

print(res)

if res > 0.5 :
    result = 1
else :
    result = 0

classes = ['REAL', 'FORGE']
classes[result]



[[0.63811666 0.6725724 ]]
0.6553444862365723


'FORGE'

# Further Development : 
# - menerapkan ke dalam GUI / web apps. GUI Preferable with TKInter or PyQt5
# - mencoba predict dengan ttd yang dilakukan secara langsung (di luar dataset)

# Note :
# - Demo : false positive and false negative Karena ttd sangat tidak konsisten , ttd dari org yg sama dapat menghasilkan ttd yang sangat berbeda. Forge kemungkinan besar terdeteksi forge, tetapi Real dapat terdeteksi sebagai forge